# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [20]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt 
import pandas as pd 
from pandas import DataFrame, Series
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import json
from pprint import pprint
import string

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [21]:
dtype_dict = {'name':str, 'review':str, 'rating':int}

with zipfile.ZipFile('amazon_baby.csv.zip', "r") as z:
    z.extractall()
products = pd.read_csv('amazon_baby.csv',dtype=dtype_dict,keep_default_na=False)


Now, let us see a preview of what the dataset looks like.

In [22]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [23]:
products.ix[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [64]:
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))

def remove_punctuation(text):
    return text.translate(remove_punct_map) 

products['review_clean'] = products['review'].apply(remove_punctuation)
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [25]:
products.ix[269]

name            The First Years Massaging Action Teether
review                          A favorite in our house!
rating                                                 5
review_clean                     A favorite in our house
Name: 269, dtype: object

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [26]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [65]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head(40)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [28]:
testIndexes = pd.read_json('module-2-assignment-test-idx.json')
test_data = products[products.index.to_series().isin(testIndexes[0])]


trainIndexes = pd.read_json('module-2-assignment-train-idx.json')
train_data = products[products.index.to_series().isin(trainIndexes[0])]

print (len(train_data))
print (len(test_data))

121232
30272


In [29]:
test_data.head()

,name,review,rating,review_clean,sentiment
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
14,Nature's Lullabies First Year Sticker Calendar,"Space for monthly photos, info and a lot of us...",5,Space for monthly photos info and a lot of use...,1
18,Nature's Lullabies Second Year Sticker Calendar,I completed a calendar for my son's first year...,4,I completed a calendar for my sons first year ...,1
24,Nature's Lullabies Second Year Sticker Calendar,Wife loves this calender. Comes with a lot of ...,5,Wife loves this calender Comes with a lot of s...,1


In [30]:
train_data.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [32]:
# instantiate a logistic regression model, and fit with X and y
sentiment_model = LogisticRegression()
fit = sentiment_model.fit(train_matrix, train_data["sentiment"])

# check the accuracy on the training set
score = sentiment_model.score(train_matrix, train_data["sentiment"])



In [33]:
sentiment_model.coef_

array([[ -3.96905723e-01,   1.37283933e-02,   6.07597207e-03, ...,
         -1.76544327e-02,   3.96474628e-06,   9.86123686e-03]])

In [34]:
coefsGreatherThanZero = sentiment_model.coef_[sentiment_model.coef_ >= 0]
print(len(coefsGreatherThanZero))
#coefs = np.array(filter(lambda x: x >0, sentiment_model.coef_))
#coefs.shape
#print (coefs)

sentiment_model.coef_.shape

79871


(1, 113305)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [37]:
weights = sentiment_model.coef_

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [38]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

64     5
82     5
102    5
Name: rating, dtype: int32


,name,review,rating,review_clean,sentiment
64,Our Baby Girl Memory Book,Really happy with this purchase. I was looking...,5,Really happy with this purchase I was looking ...,1
82,Cloth Diaper Pins Stainless Steel Traditional ...,It has been many years since we needed diaper ...,5,It has been many years since we needed diaper ...,1
102,Newborn Baby Tracker&reg; - Round the Clock Ch...,Love it love it love it!! Got my first baby t...,5,Love it love it love it Got my first baby tra...,1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [39]:
sample_test_data.iloc[0]["review"]

"Really happy with this purchase. I was looking for an english baby book (I'm French and the daddy is Flammish) and this is the perfect one. Lot of info to fill in : family, firsts...and mini pockets to preserve hair, maternity bracelet...Really happy!"

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [40]:
sample_test_data.iloc[1]['review']

"It has been many years since we needed diaper pins, but I do like to pin my socks together through the wash and have never lost one yet. I was using brass dressmaker pins but they tended to open up during the wash cycle. My wife suggested diaper pins and we tried them. Thery're working perfectly and staying together through the washing. Matter of fact they can get a little tricky to open, but of course that was the idea. Recommended to keep your socks &#34;friended up&#34;."

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [41]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print (scores)

[  9.62805184   7.67068453  10.69637887]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [1]:
pred = np.zeros(3)

def myPredict(x):
    if x > 0:
        return 1
    else: 
        return -1

for i in  range(len(scores)):
    pred[i] = myPredict(scores[i])

print(pred)

NameError: name 'np' is not defined

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [43]:
print ("Class predictions according to GraphLab Create:" )
print (sentiment_model.predict(sample_test_matrix))

Class predictions according to GraphLab Create:
[1 1 1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [44]:
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))


predProbability = np.zeros(3)

for i in  range(len(scores)):
    predProbability[i] = sigmoid(scores[i])

print(predProbability)
    

[ 0.99993415  0.99953392  0.99997737]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [45]:
print (sentiment_model.predict_proba(sample_test_matrix))

[[  6.58508846e-05   9.99934149e-01]
 [  4.66081187e-04   9.99533919e-01]
 [  2.26262122e-05   9.99977374e-01]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [74]:
predictedProbabilities =  sentiment_model.predict_proba(test_matrix)
indexes = predictedProbabilities[:,1].argsort()[::-1]
print(indexes[0:20])


[18089 28678 24233 27583 14476  9203  3976 20997 29224 22437 29450 17848
 26160 23415 19661 10504 25042  2575  8522 22751]


In [78]:
top20TestData = test_data.iloc[indexes[0:20]]
top20TestData

,name,review,rating,review_clean,sentiment
100166,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,1
158209,Ubbi Cloth Diaper Pail Liner,"(updated 3.22.13) After extensive research, tr...",5,updated 32213 After extensive research trial a...,1
133651,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,1
152013,"UPPAbaby Vista Stroller, Denny",I researched strollers for months and months b...,5,I researched strollers for months and months b...,1
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1
50735,"Joovy Zoom 360 Swivel Wheel Jogging Stroller, ...",The joovy zoom 360 was the perfect solution fo...,5,The joovy zoom 360 was the perfect solution fo...,1
21557,"Joovy Caboose Stand On Tandem Stroller, Black","Ok, I read all the reviews already posted here...",5,Ok I read all the reviews already posted here ...,1
116072,Joovy Scooter Single Stroller Greenie,"***I've posted an UPDATE at the end***First, l...",5,Ive posted an UPDATE at the endFirst let me st...,1
161127,Safety 1st Alpha Omega Elite Convertible 3-in-...,My dad just bought this car seat for me/my son...,5,My dad just bought this car seat for memy son ...,1
123632,"Zooper 2011 Waltz Standard Stroller, Flax Brown",I did a TON of research before I purchased thi...,5,I did a TON of research before I purchased thi...,1


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [80]:
reversed_arr = indexes
worst20TestData = test_data.iloc[indexes[::-1][0:20]]
worst20TestData

,name,review,rating,review_clean,sentiment
87026,Baby Einstein Around The World Discovery Center,"First off, I did manage to find this product f...",1,First off I did manage to find this product fo...,-1
120707,The European NANNY Baby Movement Monitor - EU ...,"The previous reviewers laud the ""piece of mind...",1,The previous reviewers laud the piece of mind ...,-1
2186,Philips Avent 3 Pack 9oz Bottles,"(This is a long review, but if you read the wh...",1,This is a long review but if you read the whol...,-1
10370,Wimmer-Ferguson Infant Stim-Mobile,This product should be in the hall of fame sol...,1,This product should be in the hall of fame sol...,-1
131738,"Kids Line Cascade Bow Diaper Bag, Black",I purchased this in the black color. For some...,2,I purchased this in the black color For some ...,-1
20331,Cabinet Flex-Lock (2 pack) from Safety First,"I am a mother of 4 children, ages 6-18 months....",5,I am a mother of 4 children ages 618 months I...,1
47740,"Room Magic Desk/Chair Set, Tropical Seas Natural","It's such a shame, this desk. It is marketed a...",1,Its such a shame this desk It is marketed as a...,-1
143095,"Graco ComfortSport Convertible Car Seat, Zara",This is a dreadful car seat. I would *never* r...,1,This is a dreadful car seat I would never reco...,-1
38197,JJ Cole System Bag - Graphite/Green,I just wanted to take the time to tell you how...,2,I just wanted to take the time to tell you how...,-1
145109,"Withings Smart Baby Monitor, White","Ok, the good. During the day, the quality of ...",1,Ok the good During the day the quality of the...,-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [48]:
def get_classification_accuracy(model, data, true_labels):
    predictions = model.predict(data)
    counter = 0
    
    for i  in range(len(true_labels)):
        if true_labels.iloc[i] == predictions[i]:
            counter = counter + 1 
                    
    accuracy = counter / len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [49]:
#test = test_data['sentiment']
#len(test)
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9331395348837209

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [50]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [51]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [52]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Let's see what the first example of the dataset looks like:

In [53]:
train_data.iloc[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [54]:
# instantiate a logistic regression model, and fit with X and y
simple_model = LogisticRegression()
fit = simple_model.fit(train_matrix_word_subset, train_data["sentiment"])
simple_model.score(train_matrix_word_subset, train_data["sentiment"])

0.86506038009766395

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [55]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.8688887420718816

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [56]:
simple_model.coef_

array([[ 1.36203081,  0.9256235 ,  1.18622159,  0.12446413,  0.5005022 ,
         1.51739463,  1.74372727,  0.47070728,  0.18311141,  0.0915784 ,
        -1.66313686, -0.18950968, -0.5211078 , -2.0162454 , -2.35513176,
        -0.6335363 , -0.30968213, -0.88460916, -0.33723794, -2.01824345]])

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [57]:
#simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

simpleModelCoefficients =  simple_model.coef_[::-1]
simpleModelCoefficients
#print(indexes[0:20])

array([[ 1.36203081,  0.9256235 ,  1.18622159,  0.12446413,  0.5005022 ,
         1.51739463,  1.74372727,  0.47070728,  0.18311141,  0.0915784 ,
        -1.66313686, -0.18950968, -0.5211078 , -2.0162454 , -2.35513176,
        -0.6335363 , -0.30968213, -0.88460916, -0.33723794, -2.01824345]])

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [58]:
get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])

0.966065065329286

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [59]:
get_classification_accuracy(simple_model, train_matrix_word_subset, train_data['sentiment'])

0.865060380097664

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [60]:
get_classification_accuracy(sentiment_model, test_matrix, test_data['sentiment'])

0.9331395348837209

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [61]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.8688887420718816

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [62]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print (num_positive)
print (num_negative)

101674
19558


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [63]:
num_positive / (num_negative + num_positive )

0.83867295763494787

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?